In [1]:
import os
import json
from datetime import datetime
import pandas as pd

In [2]:
# modificar segun entorno local
os.chdir("/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"
folder_gmaps = "gmaps"
folder_metadata = "metadata-sitios"

In [4]:
data_gmaps_metadata = pd.read_pickle(
    os.path.join(folder_data,folder_gmaps,folder_metadata,'gmaps_metadata_filtrado.pkl'))
data_gmaps_metadata.info()
data_gmaps_metadata.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 441868 entries, 11 to 3024974
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              441868 non-null  object 
 1   address           439290 non-null  object 
 2   gmap_id           441868 non-null  object 
 3   description       197047 non-null  object 
 4   latitude          441868 non-null  float64
 5   longitude         441868 non-null  float64
 6   category          441868 non-null  object 
 7   avg_rating        441868 non-null  float64
 8   num_of_reviews    441868 non-null  int64  
 9   price             244781 non-null  object 
 10  hours             401547 non-null  object 
 11  MISC              436358 non-null  object 
 12  state             402667 non-null  object 
 13  relative_results  371024 non-null  object 
 14  url               441868 non-null  object 
 15  estado_categoria  441868 non-null  object 
dtypes: float64(3), int64(1)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,estado_categoria
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,cajun restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...","{'Service options': ['Outdoor seating', 'Curbs...",Open ⋅ Closes 8PM,"[0x862722fdf07aed77:0xb2269b9a8a44a51b, 0x8627...",https://www.google.com/maps/place//data=!4m2!3...,Open
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...","{'Service options': ['Outdoor seating', 'Curbs...",Open ⋅ Closes 8PM,"[0x862722fdf07aed77:0xb2269b9a8a44a51b, 0x8627...",https://www.google.com/maps/place//data=!4m2!3...,Open


In [6]:
data_gmaps_metadata_hoursna = data_gmaps_metadata.dropna(subset=['hours']).copy()
data_gmaps_metadata_hoursna_explded = data_gmaps_metadata_hoursna.explode
out = data_gmaps_metadata_hoursna[data_gmaps_metadata_hoursna.assign(hours= data_gmaps_metadata_hoursna['hours'].map(tuple)).duplicated(keep=False)]

TypeError: unhashable type: 'list'